# Welcome to the Growth Rate Analysis

In this script we will analyse the growth rate $\mu$ of the total cell area in log space. Step by step (4 steps in total) you will be guided to obtain your final analysis results. Have fun 🚀

## First Step

Please enter the omero **image id** for the cultivation you want to analyze. Also provide your current **username**.

In [ ]:
# specify the omero image id
image_id = 260

# sepcify your username
username = 'root'


## Second Step

You will be asked to enter your password and we perform the analysis on the segmentation data in omero. This may take a while depending on the length of the image time-lapse. You  will get a progressbar though 😎 

In [ ]:
import getpass
import logging
from omero.gateway import BlitzGateway
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os.path as osp
import os
from tqdm.contrib.concurrent import process_map

from acia.base import Contour, Overlay
from acia.segm.omero.storer import OmeroRoISource

from acia.segm.omero.utils import list_images_in_dataset
from acia.segm.output import VideoExporter

from shapely.geometry import Polygon

def pairwise_distances(points):
    distances = []

    if len(points) == 0:
        return distances

    for a,b in zip(points, points[1:]):
        distances.append(np.linalg.norm(a-b))

    return distances

def length_and_area(contour: Contour):
    polygon = Polygon(contour.coordinates)
    #centerline = Centerline(polygon)
    length = np.max(pairwise_distances(np.array(polygon.minimum_rotated_rectangle.exterior.coords)))
    return length, polygon.area

def compute_overlay_stats(overlay: Overlay):
    cell_count = len(overlay)

    lengths = []
    areas = []
    for cont in overlay:
        length, area = length_and_area(cont)
        lengths.append(length)
        areas.append(area)

    return cell_count, lengths, areas

if __name__ == '__main__':

    basepath = osp.join('results', 'cyano')
    os.makedirs(basepath, exist_ok=True)

    serverUrl = 'omero'
    password = getpass.getpass(f'Password for {username}@{serverUrl}: ')

    omero_cred = {
        'username': username,
        'serverUrl': serverUrl,
        'password': password
    }

    result = {}

    logging.info("Connect to omero...")
    ors = OmeroRoISource(image_id, **omero_cred, scale = "MICROMETER")

    cell_counts = []
    all_lengths = []
    all_areas = []

    result = process_map(compute_overlay_stats, ors, max_workers=1, chunksize=5)
    for frame_result in result:
        cell_count, lengths, areas = frame_result

        cell_counts.append(cell_count)
        all_lengths.append(lengths)
        all_areas.append(areas)

## Third Step

Make sure your data is reasonable. We plot cell count and total cell area development over time to check.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].set_yscale('log')
ax[0].set_xlabel('Frame')
ax[0].set_ylabel('Cell Count (log-scale)')

ax[0].plot(cell_counts)

plt.title('Total cell area over time')
ax[1].set_yscale('log')
ax[1].set_xlabel('Frame')
ax[1].set_ylabel(r'Total Cell Area [$\mu m^2$]')
ax[1].grid(True)

total_frame_areas = np.array([np.sum(frame_areas) for frame_areas in all_areas])
ax[1].plot(total_frame_areas)



## Fourth Step: Select estimation range and get your results

Now you must specify the lower and upper frame where you want to estimate the growth rate. Then the growth rate will be estimated and the linear fit is visualized alongside the total log area.

In [ ]:
measure_frames = range(20, 30+1)  #measure in frames 20 to 30

In [ ]:
ln_tot_frame_areas = np.log(total_frame_areas)

linear_part = list(measure_frames)
print(linear_part)

linear_part_x = np.array(linear_part)[:,None]


from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(linear_part_x, ln_tot_frame_areas[linear_part])
print(reg.coef_)
print(reg.intercept_)
r_squared = reg.score(linear_part_x, ln_tot_frame_areas[linear_part])

growth_rates = total_frame_areas[1:] - total_frame_areas[0:-1]
plt.plot(ln_tot_frame_areas, label='log total area')
plt.plot(linear_part_x, reg.predict(linear_part_x), label='linear fit')
plt.legend()

In [ ]:
from IPython.display import display, Markdown, Latex
display(Markdown(f"""
We estimated the growth rate in frames {linear_part[0]} to {linear_part[-1]} using a [linear regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html):
- growt rate $\mu$: {reg.coef_[0]:.4f}
- Coefficient of determination: $R^2$: {r_squared:.4f}"""))